# Speech Emotion Recognition Project using an LSTM Model and RAVDESS Dataset

### Importing libraries

In [ ]:
import pandas as pd
import numpy as np

import os
import sys
import random

import librosa
import librosa.display

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, LSTM, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv1D
from keras import optimizers
from keras.optimizers import adam_v2
from keras.layers import Dropout


### Data Preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Loading dataset
data_dir = "/content/drive/My Drive/Machine Learning Stuff/speech-emotion-recognition-ravdess-data"

ravdess_directory_list = os.listdir(data_dir)
file_emotion = []
file_path = []
for dir in ravdess_directory_list:
    
    actor = os.listdir(os.path.join(data_dir,dir))
    for file in actor:
        part = file.split('.')[0]
        part = part.split('-')
        
        file_emotion.append(int(part[2]))  # third part in each file represents the emotion associated to that auido.
        file_path.append(os.path.join(data_dir,dir,file))
        
data_path = pd.DataFrame({"Emotions":file_emotion, "Path":file_path})

# changing integers to actual emotions.
data_path.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
data_path.head()


,Emotions,Path
0,neutral,/content/drive/My Drive/Machine Learning Stuff...
1,angry,/content/drive/My Drive/Machine Learning Stuff...
2,calm,/content/drive/My Drive/Machine Learning Stuff...
3,disgust,/content/drive/My Drive/Machine Learning Stuff...
4,neutral,/content/drive/My Drive/Machine Learning Stuff...


In [ ]:
# Data Augmentation to make data more complex
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def stretch(data, rate=0.85):
    return librosa.effects.time_stretch(data, rate)

def pitch(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)


In [ ]:
def extract_features(data,sample_rate):
    
    mfcc = librosa.feature.mfcc(y=data, sr=sample_rate)   # MFCC Feature extraction
    result = mfcc
    return result


def transform_audio(data, sample_rate, fns):           # function to transform audio
    fn = random.choice(fns)
    if fn == pitch:
        fn_data = fn(data, sample_rate)
    elif fn == "None":
        fn_data = data
    elif fn in [noise, stretch]:
        fn_data = fn(data)
    else:
        fn_data = data
    return fn_data


def get_features(path):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)


    #randomly transforming audio data
    fns = [noise, pitch, "None"]

    fn1_data = transform_audio(data, sample_rate, fns)
    fn2_data = transform_audio(fn1_data, sample_rate, fns)
    res1 = extract_features(fn2_data, sample_rate)
    result = [np.array(res1[:,:108])]       # data framing to fit in model

    fn1_data = transform_audio(data, sample_rate, fns)
    fn2_data = transform_audio(fn1_data, sample_rate, fns)
    res2 = extract_features(fn2_data, sample_rate)[:,:108]
    result.append(res2)

    fn1_data = transform_audio(data, sample_rate, fns)
    fn2_data = transform_audio(fn1_data, sample_rate, fns)
    res3 = extract_features(fn2_data, sample_rate)[:,:108]
    result.append(res3)

    return result

In [ ]:
X, Y = [], []
for path, emotion in zip(data_path.Path.to_list(), data_path.Emotions.to_list()):
    feature = get_features(path)
    for ele in feature:
        if ele.shape == (20, 108):
            X.append(ele)
            # appending emotion 3 times as we have made 3 augmentation techniques on each audio file.
            Y.append(emotion)
        else:
            continue

In [ ]:
X = np.array(X)
Y = np.array(Y)

In [ ]:
X.shape, Y.shape

((4275, 20, 108), (4275,))

In [ ]:
# One hot encoding y
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [ ]:
encoder.get_feature_names_out()

array(['x0_angry', 'x0_calm', 'x0_disgust', 'x0_fear', 'x0_happy',
       'x0_neutral', 'x0_sad', 'x0_surprise'], dtype=object)

In [ ]:
# Split data into training and testing data
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=21, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((3420, 20, 108), (3420, 8), (855, 20, 108), (855, 8))

In [ ]:
# Frame data to make it compatible for the model
x_train = np.expand_dims(x_train, axis=3)
x_train = np.swapaxes(x_train, 1, 2)
x_train = np.expand_dims(x_train, axis=3)

x_test = np.expand_dims(x_test, axis=3)
x_test = np.swapaxes(x_test, 1, 2)
x_test = np.expand_dims(x_test, axis=3)

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((3420, 108, 20, 1, 1), (3420, 8), (855, 108, 20, 1, 1), (855, 8))

### Model Development

In [ ]:
input_shape = x_train.shape[1:]

In [ ]:
model=Sequential()

model.add(TimeDistributed(Conv1D(16, 3, padding='same', activation='relu'),
                            input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Flatten()))
model.add(LSTM(32))
model.add(Dropout(0.2))

model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=8, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 108, 20, 1, 16)   64        
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 108, 20, 1, 16)   64        
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 108, 320)         0         
 tributed)                                                       
                                                                 
 lstm (LSTM)                 (None, 32)                45184     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                        

In [ ]:
from keras.optimizers import adam_v2
optimizer = adam_v2.Adam()
model.compile(optimizer=optimizer,
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])




In [ ]:
history = model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
27/27 [==============================] - 10s 39ms/step - loss: 2.0351 - accuracy: 0.1804 - val_loss: 2.0388 - val_accuracy: 0.1930
Epoch 2/100
27/27 [==============================] - 0s 14ms/step - loss: 1.9378 - accuracy: 0.2436 - val_loss: 1.9577 - val_accuracy: 0.2070
Epoch 3/100
27/27 [==============================] - 0s 14ms/step - loss: 1.8249 - accuracy: 0.2874 - val_loss: 1.8687 - val_accuracy: 0.2421
Epoch 4/100
27/27 [==============================] - 0s 14ms/step - loss: 1.7537 - accuracy: 0.3149 - val_loss: 1.7945 - val_accuracy: 0.2936
Epoch 5/100
27/27 [==============================] - 0s 14ms/step - loss: 1.6770 - accuracy: 0.3532 - val_loss: 1.7701 - val_accuracy: 0.2947
Epoch 6/100
27/27 [==============================] - 0s 14ms/step - loss: 1.6448 - accuracy: 0.3632 - val_loss: 1.6955 - val_accuracy: 0.3404
Epoch 7/100
27/27 [==============================] - 0s 13ms/step - loss: 1.6211 - accuracy: 0.3734 - val_loss: 1.5770 - val_accuracy: 0.3988
Epoch

In [ ]:
# predicting on test data.
pred_test = model.predict(x_test)
y_pred_emotion = encoder.inverse_transform(pred_test)

y_test_emotion = encoder.inverse_transform(y_test)

df = pd.DataFrame(columns=['Predicted Labels', 'Actual Labels'])
df['Predicted Labels'] = y_pred_emotion.flatten()
df['Actual Labels'] = y_test_emotion.flatten()

df.head(10)

27/27 [==============================] - 1s 11ms/step


,Predicted Labels,Actual Labels
0,happy,sad
1,surprise,surprise
2,surprise,surprise
3,neutral,disgust
4,sad,calm
5,fear,fear
6,calm,calm
7,calm,calm
8,calm,calm
9,calm,calm


### Model Evaluation

In [ ]:
# Classification report
print(classification_report(y_test_emotion, y_pred_emotion))

              precision    recall  f1-score   support

       angry       0.81      0.85      0.83       130
        calm       0.88      0.82      0.85       120
     disgust       0.73      0.79      0.76       105
        fear       0.70      0.79      0.74       114
       happy       0.60      0.78      0.68       102
     neutral       0.66      0.60      0.63        62
         sad       0.74      0.56      0.63       108
    surprise       0.86      0.71      0.78       114

    accuracy                           0.75       855
   macro avg       0.75      0.74      0.74       855
weighted avg       0.76      0.75      0.75       855



### Model Optimization

In [ ]:
# Increasing dropout rate to reduce overfitting, increasing number of epoches, added one hidden layer
model2=Sequential()

model2.add(TimeDistributed(Conv1D(16, 3, padding='same', activation='relu'),
                            input_shape=input_shape))
model2.add(TimeDistributed(BatchNormalization()))

model2.add(TimeDistributed(Flatten()))
model2.add(LSTM(32))
model2.add(Dropout(0.25))

model2.add(Dense(units=32, activation='relu'))
model2.add(Dropout(0.25))

model2.add(Dense(units=32, activation='relu'))
model2.add(Dropout(0.25))

model2.add(Dense(units=8, activation='softmax'))

model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_6 (TimeDis  (None, 108, 20, 1, 16)   64        
 tributed)                                                       
                                                                 
 time_distributed_7 (TimeDis  (None, 108, 20, 1, 16)   64        
 tributed)                                                       
                                                                 
 time_distributed_8 (TimeDis  (None, 108, 320)         0         
 tributed)                                                       
                                                                 
 lstm_2 (LSTM)               (None, 32)                45184     
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                      

In [ ]:
from keras.optimizers import adam_v2
optimizer = adam_v2.Adam()
model2.compile(optimizer=optimizer,
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])




In [ ]:
history = model2.fit(x_train, y_train, batch_size=100, epochs=150, validation_data=(x_test, y_test))

Epoch 1/150
35/35 [==============================] - 3s 32ms/step - loss: 2.0661 - accuracy: 0.1503 - val_loss: 2.0594 - val_accuracy: 0.1474
Epoch 2/150
35/35 [==============================] - 0s 12ms/step - loss: 2.0040 - accuracy: 0.1947 - val_loss: 1.9711 - val_accuracy: 0.2550
Epoch 3/150
35/35 [==============================] - 0s 13ms/step - loss: 1.9405 - accuracy: 0.2272 - val_loss: 1.9104 - val_accuracy: 0.2620
Epoch 4/150
35/35 [==============================] - 0s 13ms/step - loss: 1.8808 - accuracy: 0.2558 - val_loss: 1.8981 - val_accuracy: 0.2760
Epoch 5/150
35/35 [==============================] - 0s 12ms/step - loss: 1.8209 - accuracy: 0.2754 - val_loss: 1.7925 - val_accuracy: 0.3380
Epoch 6/150
35/35 [==============================] - 0s 13ms/step - loss: 1.7893 - accuracy: 0.2927 - val_loss: 1.7191 - val_accuracy: 0.3368
Epoch 7/150
35/35 [==============================] - 0s 13ms/step - loss: 1.7498 - accuracy: 0.3047 - val_loss: 1.6938 - val_accuracy: 0.3579
Epoch 

In [ ]:
from keras.optimizers import adam_v2
optimizer = adam_v2.Adam()
model.compile(optimizer=optimizer,
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])


# predicting on test data.
pred_test = model2.predict(x_test)
y_pred_emotion = encoder.inverse_transform(pred_test)

y_test_emotion = encoder.inverse_transform(y_test)

df = pd.DataFrame(columns=['Predicted Labels', 'Actual Labels'])
df['Predicted Labels'] = y_pred_emotion.flatten()
df['Actual Labels'] = y_test_emotion.flatten()

df.head(10)

27/27 [==============================] - 1s 5ms/step


,Predicted Labels,Actual Labels
0,sad,sad
1,surprise,surprise
2,surprise,surprise
3,disgust,disgust
4,neutral,calm
5,fear,fear
6,calm,calm
7,calm,calm
8,calm,calm
9,sad,calm


In [ ]:
# Classification report
print(classification_report(y_test_emotion, y_pred_emotion))

              precision    recall  f1-score   support

       angry       0.86      0.88      0.87       130
        calm       0.75      0.84      0.80       120
     disgust       0.87      0.83      0.85       105
        fear       0.61      0.74      0.67       114
       happy       0.60      0.63      0.61       102
     neutral       0.53      0.60      0.56        62
         sad       0.64      0.44      0.52       108
    surprise       0.92      0.80      0.85       114

    accuracy                           0.73       855
   macro avg       0.72      0.72      0.72       855
weighted avg       0.74      0.73      0.73       855



In [ ]:
# Save model
import pickle

In [ ]:
pickle.dump(model2, open('speech_emo_recognition.pkl', 'wb'))

In [ ]:
model.save('speech_emo_recognition.h5')